### Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from src.squlearn import Executor
from src.squlearn.encoding_circuit import ChebyshevRx
from src.squlearn.observables import IsingHamiltonian
from src.squlearn.optimizers import SGLBO
from src.squlearn.qnn import QNNRegressor, SquaredLoss


fit:   0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
executor = Executor("statevector_simulator")

# define the PQC
nqubits = 12
number_of_layers = 2
pqc = ChebyshevRx(nqubits, 1, num_layers=number_of_layers)

# define the Observable
ising_op = IsingHamiltonian(nqubits, I="S", Z="S", ZZ="S")

# define random initial parameters for the PQC and the cost operator
np.random.seed(13)
param_ini = np.random.rand(pqc.num_parameters)
param_op_ini = np.random.rand(ising_op.num_parameters)

### Optimizer options

In [ ]:
x0 = [[i * 0.02] for i in range(15)]
optimizer_options = {"bo_aqc_func": "EI", "bo_aqc_optimizer": "lbfgs", "bo_bounds": [(0.0, 0.3)], "log_file": "log_sglbo_noiseless",
                     "bo_n_calls": 30, "bo_x0_points": x0, "maxiter": 200}

### Training

In [ ]:
# define the regressor
reg = QNNRegressor(pqc, ising_op, executor, SquaredLoss(), SGLBO(optimizer_options), param_ini, param_op_ini)

# train the regressor
x_space = np.arange(0.1, 0.9, 0.1)
ref_values = np.log(x_space)

reg.fit(x_space, ref_values)

### Result

In [ ]:
# print the trained parameters for the PQC and the operator
print("Result PQC params:", reg.param)
print("Result operator params:", reg.param_op)

### Plot

In [ ]:
# plot the predicted function vs. the actual logarithm function
x = np.arange(np.min(x_space), np.max(x_space), 0.005)
y = reg.predict(x)
plt.plot(x, np.log(x), label="Tatsächliche Log-Funktion")
plt.plot(x, y, label="Vorhergesagte Funktion")

# plot the error of the QNN
plt.plot(x, np.abs(y - np.log(x)), label="Fehler")
plt.legend()
plt.show()